In [2]:
import opendatasets as od


dataset_url = "https://www.kaggle.com/code/anlthms/deepfake-example-video-clips/input"


od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Dataset URL: https://www.kaggle.com/datasets/code/anlthms


HTTPError: 403 Client Error: Forbidden for url: https://www.kaggle.com/api/v1/datasets/download/code/anlthms?raw=false

In [ ]:
# deepfake_xception_auto_split.py
import os
import cv2
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import Xception
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


root_dir = ""
real_videos_path = os.path.join(root_dir, "real")
fake_videos_path = os.path.join(root_dir, "faked")
output_root = os.path.join(root_dir, "dataset")
frames_per_video = 8
split_ratio = 0.8
img_size = (299, 299)
batch_size = 8
epochs = 10
model_path = os.path.join(root_dir, "xception_deepfake_model.h5")
# =================================================

# Enable GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
for g in gpus:
    tf.config.experimental.set_memory_growth(g, True)

# --------- Helper Function: Extract Frames ---------
def extract_frames(video_path, output_folder, frames_to_extract=8):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_count <= 0:
        print(f"[WARN] Skipping invalid video: {video_path}")
        return
    step = max(1, frame_count // frames_to_extract)
    i = 0
    frame_id = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if i % step == 0:
            frame_path = os.path.join(output_folder, f"frame_{frame_id}.jpg")
            cv2.imwrite(frame_path, frame)
            frame_id += 1
        i += 1
    cap.release()

# --------- Step 1: Create folders ---------
for split in ["train", "test"]:
    for label in ["real", "fake"]:
        os.makedirs(os.path.join(output_root, split, label), exist_ok=True)

# --------- Step 2: Split & Extract Real Videos ---------
real_videos = [v for v in os.listdir(real_videos_path) if v.endswith(".mp4")]
fake_videos = [v for v in os.listdir(fake_videos_path) if v.endswith(".mp4")]
random.shuffle(real_videos)
random.shuffle(fake_videos)

real_split = int(len(real_videos) * split_ratio)
fake_split = int(len(fake_videos) * split_ratio)

train_real = real_videos[:real_split]
test_real = real_videos[real_split:]
train_fake = fake_videos[:fake_split]
test_fake = fake_videos[fake_split:]

print(f"🎬 Total Real: {len(real_videos)} | Train: {len(train_real)} | Test: {len(test_real)}")
print(f"🎬 Total Fake: {len(fake_videos)} | Train: {len(train_fake)} | Test: {len(test_fake)}")

# --------- Step 3: Extract frames ---------
print("\n🧠 Extracting frames from Real videos...")
for v in train_real:
    extract_frames(os.path.join(real_videos_path, v),
                   os.path.join(output_root, "train", "real", v.split(".")[0]))
for v in test_real:
    extract_frames(os.path.join(real_videos_path, v),
                   os.path.join(output_root, "test", "real", v.split(".")[0]))

print("\n🧠 Extracting frames from Fake videos...")
for v in train_fake:
    extract_frames(os.path.join(fake_videos_path, v),
                   os.path.join(output_root, "train", "fake", v.split(".")[0]))
for v in test_fake:
    extract_frames(os.path.join(fake_videos_path, v),
                   os.path.join(output_root, "test", "fake", v.split(".")[0]))

print("\n✅ All frames extracted successfully!")

# --------- Step 4: Prepare ImageDataGenerators ---------
train_dir = os.path.join(output_root, "train")
test_dir = os.path.join(output_root, "test")

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)
val_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

print("✅ Data loaded successfully!")

# --------- Step 5: Build Xception Model ---------
base_model = Xception(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))
for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(1024, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()

# --------- Step 6: Train Model ---------


history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=epochs,

    verbose=1
)

# --------- Step 7: Evaluate ---------
loss, acc = model.evaluate(test_gen)
print(f"\n✅ Test Accuracy: {acc*100:.2f}% | Loss: {loss:.4f}")

# --------- Step 8: Save Model ---------
model.save(model_path)
print(f"✅ Model saved at {model_path}")



🎬 Total Real: 70 | Train: 56 | Test: 14
🎬 Total Fake: 70 | Train: 56 | Test: 14

🧠 Extracting frames from Real videos...

🧠 Extracting frames from Fake videos...

✅ All frames extracted successfully!
Found 1008 images belonging to 2 classes.
Found 252 images belonging to 2 classes.
Found 594 images belonging to 2 classes.
✅ Data loaded successfully!


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ xception (Functional)           │ (None, 10, 10, 2048)   │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,960,681 (87.59 MB)

 Trainable params: 2,099,201 (8.01 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

Epoch 1/10
 11/126 ━━━━━━━━━━━━━━━━━━━━ 3:56 2s/step - accuracy: 0.5067 - loss: 0.7178

In [ ]:

model = Xception(weights='imagenet', include_top=True)


model.save("xception_deepfake_models.h5")

